In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [2]:
df = pd.read_csv('last_df.csv')

In [3]:
df.head(1)

,Price,Num_of_rooms,Total_area,Living_space,Balcony,Bathroom,Windows,Sale_method,House_type,WMO,...,расстояние_до_обводный_канал,расстояние_до_волковская,расстояние_до_бухарестская,расстояние_до_международная,расстояние_до_дунайская,расстояние_до_проспект_славы,расстояние_до_шушары,group,Subway_station,Time_for_subway
0,9800000,3.0,73,NaN,балкон,раздельный,"во двор, на улицу, на солнечную сторону",свободная,кирпичный,Шушары,...,23.334,21.026,19.195,17.306,12.66,15.105,9.662,1.0,no_info,no_info


Посоветовавшись с риэлтором, который выступал в качестве эксперта было принято решение также удалить фичу 'Sale_method'

In [4]:
df = df.drop('Sale_method', axis=1)

### Вытаскиваем уникальные значения из некоторых категориальных фичей

In [5]:
# преобразуем категориальные признаки.
def convert_lists(df: pd.DataFrame) -> pd.DataFrame:
    cols = [
        "Balcony", "Bathroom", 
        "Windows", "House_type"
           ]
    new_cols = []
    for col in cols:
        series = df.loc[:, col]
        data = []
        unique_values = set()
        new_cols_data = []

        for row in series.values:
            row_c = [value.strip() for value in (row.split(",") if isinstance(row, str) else [])]
            for element in row_c:
                if not element in unique_values:
                    unique_values.add(element)
            data.append(row_c)
    
        unique_values = list(unique_values)

        for row in data:
            new_cols_data.append([1 if value in row else 0 for value in unique_values])
    
        col_names = [f"{col}_{i+1}" for i in range(len(unique_values))]
        new_cols.append(pd.DataFrame(data=new_cols_data, columns=col_names, dtype=np.int8))
        print("\n".join(map(str,zip(unique_values, col_names))))

        df.drop(columns=[col], axis=1, inplace=True)

    return pd.concat([df]+new_cols, axis=1)

df = convert_lists(df)

('балкон', 'Balcony_1')
('лоджия', 'Balcony_2')
('совмещенный', 'Bathroom_1')
('раздельный', 'Bathroom_2')
('на улицу', 'Windows_1')
('на солнечную сторону', 'Windows_2')
('во двор', 'Windows_3')
('монолитно-кирпичный', 'House_type_1')
('монолитный', 'House_type_2')
('кирпичный', 'House_type_3')
('панельный', 'House_type_4')
('блочный', 'House_type_5')


In [6]:
num_features = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]
cat_features = [cname for cname in df.columns if df[cname].dtype == "object"]

In [7]:
cat_features

['WMO', 'name', 'Subway_station', 'Time_for_subway']

In [8]:
df['Time_for_subway']

0       no_info
1            15
2            10
3            31
4            10
         ...   
1452    no_info
1453    no_info
1454    no_info
1455    no_info
1456    no_info
Name: Time_for_subway, Length: 1457, dtype: object

In [9]:
df['Subway_station'].unique()

array(['no_info', 'Гостиный двор', 'Комендантский проспект', 'Приморская',
       'Звёздная', 'Международная', 'Чернышевская', 'Проспект Ветеранов',
       'Владимирская', 'Электросила', 'Автово', 'Удельная',
       'Проспект Просвещения', 'Лесная', 'Проспект Большевиков',
       'Ладожская', 'Площадь Александра Невского 1', 'Площадь Мужества',
       'Беговая', 'Новочеркасская', 'Обухово', 'Пионерская',
       'Технологический институт 2', 'Академическая', 'Петроградская',
       'Московская', 'Площадь Александра Невского 2', 'Парнас', 'Озерки',
       'Волковская', 'Нарвская', 'Крестовский остров', 'Чкаловская',
       'Дунайская', 'Улица Дыбенко', 'Девяткино', 'Гражданский проспект',
       'Невский проспект', 'Рыбацкое', 'Горьковская',
       'Технологический институт', 'Фрунзенская', 'Елизаровская',
       'Спортивная', 'Чёрная речка', 'Купчино', 'Василеостровская',
       'Обводный канал', 'Балтийская', 'Спасская', 'Лиговский проспект',
       'Бухарестская', 'Садовая', 'Ленински

Воспользуем Label encoder, чтобы разобраться со столбцом 

In [10]:
categorical_feature_mask = df.dtypes==object
categorical_cols = df.columns[categorical_feature_mask].tolist()

In [11]:
le = LabelEncoder()
df[cat_features] = df[cat_features].apply(lambda col: le.fit_transform(col))

In [12]:
df['name'].isnull().sum()

0

### Заполняем оставшиеся пропуски

In [13]:
# в жилой площади пропуски заменим на среднее значение по столбцу
df['Living_space'] = df['Living_space'].fillna(df['Living_space'].mean())

In [14]:
# name и group вместо пропусков поставим значение 0
df[['name', 'group']] = df[['name','group']].fillna(0)

In [15]:
df.describe()

,Price,Num_of_rooms,Total_area,Living_space,WMO,name,log_price,Price_m_2,Price_m_2_cat,lat_object,...,Bathroom_1,Bathroom_2,Windows_1,Windows_2,Windows_3,House_type_1,House_type_2,House_type_3,House_type_4,House_type_5
count,1.457000e+03,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,...,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000,1457.000000
mean,2.567371e+07,2.514070,96.301990,52.895850,58.205216,9.550446,0.617454,253772.834904,2.260810,59.941930,...,0.321894,0.445436,0.402883,0.110501,0.415237,0.079616,0.409746,0.448181,0.059025,0.003432
std,2.492439e+07,1.692331,74.772857,47.669105,29.023811,5.789018,0.864135,92267.871139,1.324664,0.077373,...,0.467363,0.497184,0.490646,0.313621,0.492932,0.270790,0.491956,0.497478,0.235753,0.058500
min,1.925000e+06,0.000000,10.000000,5.000000,0.000000,0.000000,-1.612979,107428.571429,0.000000,59.573957,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.710763e+06,1.000000,47.000000,26.300000,32.000000,4.000000,0.005329,183782.882583,1.000000,59.903877,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.890000e+07,3.000000,83.000000,48.500000,59.000000,11.000000,0.671256,235527.698211,2.000000,59.947176,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.250000e+07,3.000000,125.000000,61.600000,84.000000,14.000000,1.213335,301431.560272,4.000000,59.989090,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,3.300000e+08,10.000000,1024.000000,1000.000000,103.000000,18.000000,3.531187,534304.806500,4.000000,60.194562,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Нормализация данных

In [16]:
# columns = df.columns

In [17]:
# #Будем использовать метод normalize
# normalize = preprocessing.normalize(df)
# df = pd.DataFrame(normalize, columns=columns)

In [18]:
df.to_csv('clean.csv', index=False)